# Build a basic model

This _Build a basic model_ tutorial illustrates how to build a simple D-Flow FM model from scratch using [HYDROLIB-core](https://github.com/Deltares/HYDROLIB-core). 

### Import HYDROLIB-core functionalities and setup D-Flow FM model
The commands below import all needed packages and will prepare a model that will be saved in a subdir `testmodel/` within the current working dir.

In [ ]:
from hydrolib.core.io.structure.models import FlowDirection, StructureModel, Weir
from hydrolib.core.io.mdu.models import FMModel
from pathlib import Path

print("Package import successful!")

In [ ]:
workdir = Path("testmodel")
mdufile = "test.mdu"

fm = FMModel()
fm.filepath = workdir / mdufile

Add a hydraulic structure; note that this is invalid because the model does not 
have a 1D network with branches and coordinates yet, but it will work for demo purposes 
and the network may be added later.

In [ ]:
# Add weir to the structure set/model and add this structure set to D-Flow FM model 
struc = Weir(branchId='someBranch', chainage = 123.0, allowedflowdir=FlowDirection.none, crestlevel=0.0)
struc.comments.crestlevel = "This is a comment"
fm.geometry.structurefile = [StructureModel(structure=[struc])]

Note that the creation of a `Weir` and other model objects requires several input arguments.
A `ValidationError` will be raised when the model is invalid or incomplete.
For instance, in the above example, if the `StructureModel` had been assigned directly to `structurefile` instead of as a list, that would have triggered a `ValidationError`.

### Setup a DIMR model
Now let's add this model to an integrated model (DIMR config) and save it.

In [ ]:
from hydrolib.core.io.dimr.models import DIMR, FMComponent

dimr = DIMR()
dimr.component.append(
    FMComponent(name="test", workingDir=workdir, inputfile=mdufile, model=fm)
)

dimr_config = Path("dimr_config.xml")
dimr.save(filepath=dimr_config, recurse=True)


### The model files on disk
Now, let's see which model input files were saved:

In [ ]:
dimr.show_tree()

The save call on the highest-level DIMR object will result in recursive saves of all child models in the model hierarchy, so this results in four files (`dimr_config.xml`, `network.nc`, `structures.ini`,`test.mdu`) in the working directory.
Some more in-depth background about recursive saving of a model tree is given in another tutorial: [Loading and saving](loading_and_saving_a_model.md).